In [ ]:
!pip install praw
import pandas as pd
import praw
import warnings
import logging

warnings.filterwarnings("ignore", category=UserWarning, module="praw")

logging.getLogger("praw").setLevel(logging.ERROR)


reddit = praw.Reddit(
    client_id="#######",
    client_secret="##########",
    user_agent="############",
)

MLsubreddit = reddit.subreddit("MachineLearning")


##  Post-level: id, title, author, score, num_comments, created_utc, selftext,subreddit
## Comment-level: id, author, body, score, created_utc, parent_id, link_id

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 9.1 MB/s eta 0:00:00


In [ ]:

subs = MLsubreddit.top('year')
for submission in subs:
    print(submission.title)

    print(submission.score)

    print(submission.id)

    print(submission.url)


/tmp/ipython-input-2265198772.py:1: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  subs = MLsubreddit.top('year')


[D] The "it" in AI models is really just the dataset?
1309
1cjxh9u
https://i.redd.it/uadactn53eyc1.png
[N] 2024 Nobel Prize for Physics goes to ML and DNN researchers J. Hopfield and G. Hinton
1162
1fywi9h
https://www.reddit.com/r/MachineLearning/comments/1fywi9h/n_2024_nobel_prize_for_physics_goes_to_ml_and_dnn/
[D] Why do PhD Students in the US seem like overpowered final bosses 
1121
1g7dzkp
https://www.reddit.com/r/MachineLearning/comments/1g7dzkp/d_why_do_phd_students_in_the_us_seem_like/
[D]Stuck in AI Hell: What to do in post LLM world
839
1h7jg87
https://www.reddit.com/r/MachineLearning/comments/1h7jg87/dstuck_in_ai_hell_what_to_do_in_post_llm_world/
[D] The winner of the NeurIPS 2024 Best Paper Award  sabotaged the other teams
707
1hctf36
https://www.reddit.com/r/MachineLearning/comments/1hctf36/d_the_winner_of_the_neurips_2024_best_paper_award/
[D] Can we please stop using "is all we need" in titles?
702
1hlbtrs
https://www.reddit.com/r/MachineLearning/comments/1hlbtrs/d_can_

In [ ]:
subs_top = list(MLsubreddit.top('year', limit=1000))
print(len(subs_top))
#max 993

/tmp/ipython-input-2207082960.py:1: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  subs_top = list(MLsubreddit.top('year', limit=1000))


993


In [ ]:
subs_hot = list(MLsubreddit.hot(limit=1000))
print(len(subs_hot))
#max 169

for sub in subs_hot:
  print(sub.title , " ",sub.score)

180
[D] Simple Questions Thread   5
[D] Self-Promotion Thread   2
[D] People in ML/DS/AI field since 5-10 years or more, are you tired of updating yourself with changing tech stack?   37
[P] Small and Imbalanced dataset - what to do   18
Problem with dataset for my my physics undergraduate paper. Need advice about potential data leakage. [N]   3
[R] Code for Flow Stochastic Segmentation Networks (ICCV 20205)   2
[P] Can I use test set reviews to help predict ratings, or is that cheating?   2
custom Vulkan C++ machine learning library vs TensorFlow [R]   1
[D] Best way to partition longitudinal data into pre and post time periods for predictive model?   3
[2507.17338] Mobile Manipulation with Active Inference for Long-Horizon Rearrangement Tasks   1
[D] Got Spare Time – What’s Worth Doing?   36
[P] Sensor calibration correction   7
[R] Fuzzy-Pattern Tsetlin Machine   37
[D] Google DeepMind Analytics Engineer Interview Prep   16
[R] Position: The Current AI Conference Model is Unsustaina

In [ ]:

subs_controversial= list(MLsubreddit.controversial(limit=1000))
print(len(subs_controversial))
#max 996

996


In [ ]:
pd.set_option('max_colwidth', None)

def submissions_to_df(submissions):
    col = ['id', 'title', 'author', 'score', 'num_comments', 'created_utc', 'selftext', 'subreddit']
    data = []
    for sub in submissions:
        data.append([
            sub.id,
            sub.title,
            sub.author,
            sub.score,
            sub.num_comments,
            sub.created_utc,
            sub.selftext,
            sub.subreddit
        ])
    return pd.DataFrame(data, columns=col)

# from submissions to dataframes will be used for analysis, visualizations later
df_top_posts= submissions_to_df(subs_top)
df_hot_posts = submissions_to_df(subs_hot)
df_controversial_posts = submissions_to_df(subs_controversial)



In [ ]:
# adding this source col so i can get them back clean to its original state without losing the source
df_top_posts['source'] = 't'
df_hot_posts['source'] = 'h'
df_controversial_posts['source'] = 'c'

In [ ]:
################ took 15 min for execution take care ######################################
def get_comments_for_submission(submission):

    submission.comment_sort = "top"
    submission.comment_limit = 10     # only getting the first 10 comments sorting type(hot)
    submission.comments.replace_more(limit=0) # dont traverse through the whole comments tree
    comments = submission.comments[:10]
    return sorted(comments, key=lambda x: x.score, reverse=True)


def get_comments_df(submissions):

    all_comments = []
    for sub in submissions:
        comments = get_comments_for_submission(sub)
        for c in comments:
            all_comments.append([
                c.id,
                str(c.author),
                c.body,
                c.score,
                c.created_utc,
                c.parent_id,
                c.link_id
            ])
    return pd.DataFrame(all_comments, columns=[
        'id', 'author', 'body', 'score', 'created_utc', 'parent_id', 'link_id'
    ])



df_top_comments = get_comments_df(subs_top)



In [ ]:
df_hot_comments = get_comments_df(subs_hot)

In [ ]:
df_controversial_comments = get_comments_df(subs_controversial)

TooManyRequests: received 429 HTTP response

In [ ]:
# adding this so i can combine them back clean
df_top_comments['source'] = 't'
df_hot_comments['source'] = 'h'
df_controversial_comments['source'] = 'c'

#Some insights

### all posts count is 2159
### all comments count is 9046

In [ ]:
#concatenation for cleaning and profiling

df_all_posts = pd.concat(
    [df_top_posts, df_hot_posts, df_controversial_posts],
    ignore_index=True
)


df_all_comments = pd.concat(
    [df_top_comments, df_hot_comments, df_controversial_comments],
    ignore_index=True
)


In [ ]:
df_all_posts.head()

In [ ]:
'''
#using spark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType ,DoubleType
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("author", StringType(), True),
    StructField("score", IntegerType(), True),
    StructField("num_comments", IntegerType(), True),
    StructField("created_utc", DoubleType(), True),
    StructField("selftext", StringType(), True),
    StructField("subreddit", StringType(), True)
])

df_spark_all_posts_bronze = spark.createDataFrame(df_all_posts, schema=schema)
df_spark_all_posts_bronze.printSchema()'''


#Data Profiling And Cleaning

In [ ]:
df_all_posts_silver =df_all_posts.copy()
df_all_posts_silver.columns= ["post_id","title","author","score","num_comments","created_at","post_content","subreddit","source"]

In [ ]:
df_all_posts_silver.head()

In [ ]:
df_all_posts_silver.describe()

In [ ]:
for col in df_all_posts_silver.columns:
    print(col, df_all_posts_silver[col].dtype)


In [ ]:
#changing obj cols into strings to avoid some potential errors
object_cols = df_all_posts_silver.select_dtypes(include='object').columns
df_all_posts_silver[object_cols] = df_all_posts_silver[object_cols].astype('string')

#changing created_at col type from float into datetime
df_all_posts_silver['created_at'] = pd.to_datetime(df_all_posts_silver['created_at'], unit='s').dt.date


#df_all_posts_silver['created_at'] = df_all_posts_silver['created_at'].dt.floor('s')


In [ ]:
for col in df_all_posts_silver.columns:
    print(col, df_all_posts_silver[col].dtype)


In [ ]:
df_all_posts_silver.head()

In [ ]:
df_all_posts_silver.isnull().sum()

In [ ]:
# replacing nulls with Unknown avoidin potential errors
df_all_posts_silver['author'] = df_all_posts_silver['author'].fillna('Unknown')
df_all_posts_silver.isnull().sum()

In [ ]:
df_all_posts_silver.apply(lambda x: x.duplicated().sum())

In [ ]:
df_all_posts_silver = df_all_posts_silver.drop_duplicates(subset=['post_id'], keep='first')

In [ ]:
df_all_posts_silver.apply(lambda x: x.duplicated().sum())
#there was 9 post_id dups and 9 created_at dups before 2159 and after 2150

In [ ]:
df_all_posts_silver = df_all_posts_silver[df_all_posts_silver['post_content'].str.strip() != '']
#removing any empty posts before 2150 and after 1787


In [ ]:
df_all_posts_silver.describe()

In [ ]:
df_all_posts_silver.apply(lambda x: x.duplicated().sum())
#there was 375 post_content dups and became 13 after

In [ ]:
#library for detecting english and eliminating other posts
!pip install langdetect

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df_all_posts_silver = df_all_posts_silver[df_all_posts_silver['post_content'].apply(is_english)]


In [ ]:
df_all_posts_silver['title'] = df_all_posts_silver['title'].str.replace(r'^\[[^\]]*\]\s*', '', regex=True)
print(df_all_posts_silver.count())
# row count before english filtering 1786 and after 1774

df_all_posts_silver.head()

# starting to clean comments table same what we did with posts

In [ ]:
df_all_comments.head()

In [ ]:
df_all_comments_silver =df_all_comments.copy()
df_all_comments_silver.columns= ["comment_id","author","comment_content","score","created_at","parent_id","post_id","source"]

In [ ]:
df_all_comments_silver.describe()

In [ ]:
for col in df_all_comments_silver.columns:
    print(col, df_all_comments_silver[col].dtype)


In [ ]:
#changing obj cols into strings
object_cols = df_all_comments_silver.select_dtypes(include='object').columns
df_all_comments_silver[object_cols] = df_all_comments_silver[object_cols].astype('string')

#changing created_at col type from float into datetime
df_all_comments_silver['created_at'] = pd.to_datetime(df_all_comments_silver['created_at'], unit='s').dt.date

#df_all_comments_silver['created_at'] = df_all_comments_silver['created_at'].dt.floor('s')



In [ ]:
#Note: dropping the parent_id col as its the same as the post_id col because i only got the comment not their replies so there is no hiearchy there that's why they are both the same
df_all_comments_silver.drop('parent_id', axis=1, inplace=True)

for col in df_all_comments_silver.columns:
    print(col, df_all_comments_silver[col].dtype)

In [ ]:
df_all_comments_silver.isnull().sum()

In [ ]:
df_all_comments_silver.apply(lambda x: x.duplicated().sum())

In [ ]:
df_all_comments_silver = df_all_comments_silver.drop_duplicates(subset=['comment_id'], keep='first')
df_all_comments_silver.apply(lambda x: x.duplicated().sum())
#before 9046 after 9005

In [ ]:
df_all_comments_silver = df_all_comments_silver[df_all_comments_silver['comment_content'].str.strip() != '']
#only one comment was deleted

In [ ]:
#df_all_comments_silver = df_all_comments_silver[df_all_comments_silver['comment_content'].apply(is_english)]
# 703 comment was dropped (assumed it wasn't english because ITS NOT GOOD WITH EMOJIS AND SMALL TEXT)

In [ ]:
df_all_comments_silver.count()

In [ ]:
!pip install fasttext
!wget -O lid.176.ftz https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

In [ ]:
# lib provided meta that handle text efficiently mostly used for social media
import fasttext

model = fasttext.load_model("lid.176.ftz")

def is_english_fasttext(text):
    try:
        prediction = model.predict(text, k=1)
        language = prediction[0][0].replace("__label__", "")
        return language == "en"
    except:
        return False


In [ ]:
df_english = df_all_posts_silver[df_all_posts_silver['post_content'].apply(is_english_fasttext)]
#nothing was dropped all english

In [ ]:
df_all_comments_silver.count()

In [ ]:
############### separating the data for potential specific analysis #################

df_top_posts_cleaned = df_all_posts_silver[df_all_posts_silver['source'] == 't']

df_hot_posts_cleaned = df_all_posts_silver[df_all_posts_silver['source'] == 'h']

df_controversial_posts_cleaned = df_all_posts_silver[df_all_posts_silver['source'] == 'c']

In [ ]:
# dropping the source col we don't need it anymore
'''
df_top_posts_cleaned.drop('source', axis=1, inplace=True)
df_hot_posts_cleaned.drop('source', axis=1, inplace=True)
df_controversial_posts_cleaned.drop('source', axis=1, inplace=True)
'''

In [ ]:
df_all_posts_silver.to_parquet("all_cleaned_posts.parquet", index=False)
df_all_comments_silver.to_parquet("all_cleaned_comments.parquet", index=False)